<a href="https://colab.research.google.com/github/WASasquatch/portrait-generator-init-image-prep/blob/main/Portrait_Generator_init_image_Prep_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portrait Generator `init_image` Prep v1.0 ![visitors](https://visitor-badge.glitch.me/badge?page_id=portrait-generator-prep-images&left_color=blue&right_color=orange) 
[![GitHub](https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white)](https://github.com/WASasquatch/portrait-generator-init-image-prep)

Process portrait images to be more compatible with the Portrait Generator in [Disco Diffusion](https://discodiffusion.com/) or [DD Portrait Playground](https://rebrand.ly/portrait-playground) for creating unique portraits.

## Why prep portrait `init_images`?

Prepping portrait images allows DD to start with an initial init that is easier to imrpovise on. When using a raw portrait of a person, DD has a hard time keeping the cohesion of the original image through diffusion.

In [ ]:
#@title Setup Environment
!mkdir -p raw_images result_images
!pip install -q wget Pillow ipyplot

import os, wget, re, cv2, ipyplot, shutil
import numpy as np
import ipywidgets as widgets
from os import walk
from pprint import pprint
from PIL import ImageFilter
from PIL import Image as PILImage 
from IPython.display import display, Javascript, clear_output
from google.colab.output import eval_js
from base64 import b64encode
from google.colab import files

do_sample_images = True #@param{type: 'boolean'}
sample_images = ['https://unsplash.com/photos/3TLl_97HNJo/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8MXx8cG9ydHJhaXR8ZW58MHx8fHwxNjU5Mjg4OTg4&force=true', 'https://unsplash.com/photos/bqe0J0b26RQ/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8M3x8cG9ydHJhaXR8ZW58MHx8fHwxNjU5Mjg4OTg4&force=true', 'https://unsplash.com/photos/iFgRcqHznqg/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8MTB8fHBvcnRyYWl0fGVufDB8fHx8MTY1OTI4ODk4OA&force=true', 'https://unsplash.com/photos/AGlO2jlVE4c/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8NHx8cG9ydHJhaXQlMjBibGFjayUyMG1hbnxlbnwwfHx8fDE2NTkyOTQ4MzM&force=true']
sample_path = '.'+os.sep+'raw_images'
result_path = '.'+os.sep+'result_images'
maximum_image_width = 512 #@param{type: 'number'}
#@markdown **Note:** `maximum_image_width` controls the maximum allowed size of returned sample images as well as your uploaded images if `resize_uploaded_images` is `True`
valid_ext = ['.jpg','.jpeg','.gif','.png']

class log:
    f = lambda color: lambda string: print(color + string + "\33[0m")
    black = f("\33[30m")
    red = f("\33[31m")
    green = f("\33[32m")
    yellow = f("\33[33m")
    blue = f("\33[34m")
    megenta = f("\33[35m")
    cyan = f("\33[36m")
    white = f("\33[37m")
    bold = f("\033[1m")
    boldred = f("\033[1m\33[31m")
    boldgreen = f("\033[1m\33[32m")
    boldyellow = f("\033[1m\33[33m")
    boldblue = f("\033[1m\33[34m")
    boldmegenta = f("\033[1m\33[35m")
    boldcyan = f("\033[1m\33[36m")

def resize_source_image(image, width):
  img = PILImage.open(image)
  wpercent = (width / float(img.size[0]))
  hsize = int((float(img.size[1]) * float(wpercent)))
  img = img.resize((width, hsize), PILImage.ANTIALIAS)
  img.save(image)

def imshow(name, img):
  img = cv2.imread(img)
  js = Javascript('''
  async function showImage(name, image, width, height) {
    img = document.getElementById(name);
    if(img == null) {
      img = document.createElement('img');
      img.id = name;
      document.body.appendChild(img);
    }
    img.src = image;
    img.width = width / 2;
    img.height = height / 2;
    img.style = "margin: 5px";
  }
  ''')
  height, width = img.shape[:2]
  ret, data = cv2.imencode('.jpg', img)
  data = b64encode(data)
  data = data.decode()
  data = 'data:image/jpg;base64,' + data
  display(js)
  eval_js(f'showImage("{name}", "{data}", {width}, {height})')  

if sample_images:
  i = 1
  for img in sample_images:
    download = wget.download(img, out = sample_path)
    file_ext = os.path.splitext(download)[1]
    new_file = sample_path+os.sep+'sample_'+str(i)+file_ext
    os.rename(download, new_file)
    resize_source_image(new_file, maximum_image_width)
    i = i+1

log.boldgreen("Setup complete.")

# Upload Image

Open the file manager on the left hand side, and upload your files to the `raw_images` folder. If the `raw_images` folder doesn't exist, run the **Setup Environment** cell. 

In [ ]:
#@title Unpack Zip Arvhives

raw_filenames = next(walk(sample_path), (None, None, []))[2]
for rf in raw_filenames:
  file_ext = os.path.splitext(rf)[1]
  if file_ext.lower() == '.zip':
    log.yellow('Extracting archive '+rf)
    extract = shutil.unpack_archive(sample_path+os.sep+rf, sample_path)
    log.boldgreen(' - Extracted archive successfully')


# Image Settings

In [ ]:
gaussian_blur_radius = 5 #@param{type: 'number'}
gaussian_noise_amount = 1 #@param{type: 'number'}
gaussian_noise_blur_radius = 1.5 #@param{type: 'number'}
resize_uploaded_images = True #@param{type: 'boolean'}
show_processed_images = True #@param{type: 'boolean'}


# Process Image(s)

In [ ]:
#@title Process Images
filenames = next(walk(sample_path), (None, None, []))[2]

if filenames:
  for img in sorted(filenames):
    file_ext = os.path.splitext(img)[1]
    if file_ext in valid_ext:
      log.boldblue(f"Processing: {img}")
      image_raw_path = sample_path+os.sep+img
      if resize_uploaded_images:
        resize_source_image(image_raw_path, maximum_image_width)
      image_result_path = result_path+'/result_'+img
      im1 = PILImage.open(image_raw_path) 
      im2 = im1.filter(ImageFilter.GaussianBlur(radius = gaussian_blur_radius))
      im2.save(image_result_path)
      print(' - Gaussian Blur applied')
      im3 = cv2.imread(image_result_path)
      gn = np.random.normal(0,gaussian_noise_amount,im3.size)
      gn = gn.reshape(im3.shape[0],im3.shape[1],im3.shape[2]).astype('uint8')
      gi = cv2.add(im3,gn)
      cv2.imwrite(image_result_path, gi)
      print(' - Gaussian Noise applied')
      im1 = PILImage.open(image_result_path) 
      im2 = im1.filter(ImageFilter.GaussianBlur(radius = gaussian_noise_blur_radius))
      im2 = im2.filter(ImageFilter.SHARPEN)
      im2.save(image_result_path)
      print(' - Gaussian Noise Blur applied')
      log.boldgreen(' - Complete')

print("")
log.boldgreen('All images processed.')

if show_processed_images:

  clear_output()

  filenames = next(walk(result_path), (None, None, []))[2]

  print("")
  log.boldblue("Showing Processed Images:")
  print("")

  for img in filenames:
    file_ext = os.path.splitext(img)[1]
    if file_ext in valid_ext:
      img_path = result_path+os.sep+img
      imshow(img, img_path)



# Archive Results

In [ ]:
download_filename = 'portrait_prep_result_images' #@param{type: 'string'}
shutil.make_archive(download_filename, 'zip', result_path)
files.download('.'+os.sep+download_filename+'.zip')